<a href="https://colab.research.google.com/github/olaviinha/SloppyButchery/blob/main/Autotune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Autotune <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Sloppy Butchery</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/SloppyButchery" target="_blank"><font color="#999" size="4">Github</font></a>

#### Autotune audio file or a directory of audio files to:
- scale
- nearest note
- another audio file
- [Chords Guru Turbo 100a Deluxe](https://ki.gy/cv) chord progression

#### Tips:
- All directory/file paths should be relative to your Google Drive root (e.g. `audio/voice/vocals.wav` if you have a directory called _audio_ in your drive, etc.)
- Try 44.1 kHz WAV if you experience problems with other formats.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>This notebook requires moutning Google Drive.</small>

force_setup = False
repositories = []
pip_packages = 'librosa soundfile scipy psola mido'
apt_packages = ''
mount_drive = True #@ param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive == True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e ./{install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import time, sys
from datetime import timedelta
















# og autotune.py

from functools import partial
from pathlib import Path
import argparse
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.signal as sig
import psola


SEMITONES_IN_OCTAVE = 12

def degrees_from(scale: str):
    """Return the pitch classes (degrees) that correspond to the given scale"""
    degrees = librosa.key_to_degrees(scale)
    # To properly perform pitch rounding to the nearest degree from the scale, we need to repeat
    # the first degree raised by an octave. Otherwise, pitches slightly lower than the base degree
    # would be incorrectly assigned.
    degrees = np.concatenate((degrees, [degrees[0] + SEMITONES_IN_OCTAVE]))
    return degrees


def closest_pitch(f0):
    """Round the given pitch values to the nearest MIDI note numbers"""
    midi_note = np.around(librosa.hz_to_midi(f0))
    # To preserve the nan values.
    nan_indices = np.isnan(f0)
    midi_note[nan_indices] = np.nan
    # Convert back to Hz.
    return librosa.midi_to_hz(midi_note)


def closest_pitch_from_scale(f0, scale):
    """Return the pitch closest to f0 that belongs to the given scale"""
    # Preserve nan.
    if np.isnan(f0):
        return np.nan
    degrees = degrees_from(scale)
    midi_note = librosa.hz_to_midi(f0)
    # Subtract the multiplicities of 12 so that we have the real-valued pitch class of the
    # input pitch.
    degree = midi_note % SEMITONES_IN_OCTAVE
    # Find the closest pitch class from the scale.
    degree_id = np.argmin(np.abs(degrees - degree))
    # Calculate the difference between the input pitch class and the desired pitch class.
    degree_difference = degree - degrees[degree_id]
    # Shift the input MIDI note number by the calculated difference.
    midi_note -= degree_difference
    # Convert to Hz.
    return librosa.midi_to_hz(midi_note)


def aclosest_pitch_from_scale(f0, scale):
    """Map each pitch in the f0 array to the closest pitch belonging to the given scale."""
    sanitized_pitch = np.zeros_like(f0)
    for i in np.arange(f0.shape[0]):
        sanitized_pitch[i] = closest_pitch_from_scale(f0[i], scale)
    # Perform median filtering to additionally smooth the corrected pitch.
    smoothed_sanitized_pitch = sig.medfilt(sanitized_pitch, kernel_size=11)
    # Remove the additional NaN values after median filtering.
    smoothed_sanitized_pitch[np.isnan(smoothed_sanitized_pitch)] = \
        sanitized_pitch[np.isnan(smoothed_sanitized_pitch)]
    return smoothed_sanitized_pitch


def autotune(audio, sr, correction_function, plot=False):
    # Set some basis parameters.
    frame_length = 2048
    hop_length = frame_length // 4
    fmin = librosa.note_to_hz('C2')
    fmax = librosa.note_to_hz('C7')

    # Pitch tracking using the PYIN algorithm.
    f0, voiced_flag, voiced_probabilities = librosa.pyin(audio,
                                                         frame_length=frame_length,
                                                         hop_length=hop_length,
                                                         sr=sr,
                                                         fmin=fmin,
                                                         fmax=fmax)

    # Apply the chosen adjustment strategy to the pitch.
    corrected_f0 = correction_function(f0)

    if plot:
        # Plot the spectrogram, overlaid with the original pitch trajectory and the adjusted
        # pitch trajectory.
        stft = librosa.stft(audio, n_fft=frame_length, hop_length=hop_length)
        time_points = librosa.times_like(stft, sr=sr, hop_length=hop_length)
        log_stft = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(log_stft, x_axis='time', y_axis='log', ax=ax, sr=sr, hop_length=hop_length, fmin=fmin, fmax=fmax)
        fig.colorbar(img, ax=ax, format="%+2.f dB")
        ax.plot(time_points, f0, label='original pitch', color='cyan', linewidth=2)
        ax.plot(time_points, corrected_f0, label='corrected pitch', color='orange', linewidth=1)
        ax.legend(loc='upper right')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [M:SS]')
        plt.savefig('pitch_correction.png', dpi=300, bbox_inches='tight')

    # Pitch-shifting using the PSOLA algorithm.
    return psola.vocode(audio, sample_rate=int(sr), target_pitch=corrected_f0, fmin=fmin, fmax=fmax)













# Additions to autotune functions

def closest_pitch_from_notes(f0, notes_string):
    """Return the pitch closest to f0 that belongs to the given scale"""
    # Preserve nan.
    if np.isnan(f0):
        return np.nan
    degrees = notes_string_to_degrees(notes_string)
    midi_note = librosa.hz_to_midi(f0)
    # Subtract the multiplicities of 12 so that we have the real-valued pitch class of the
    # input pitch.
    degree = midi_note % SEMITONES_IN_OCTAVE
    # Find the closest pitch class from the scale.
    degree_id = np.argmin(np.abs(degrees - degree))
    # Calculate the difference between the input pitch class and the desired pitch class.
    degree_difference = degree - degrees[degree_id]
    # Shift the input MIDI note number by the calculated difference.
    midi_note -= degree_difference
    # Convert to Hz.
    return librosa.midi_to_hz(midi_note)

def aclosest_pitch_from_notes(f0, notes_string):
    """Map each pitch in the f0 array to the closest pitch belonging to the given scale."""
    sanitized_pitch = np.zeros_like(f0)
    for i in np.arange(f0.shape[0]):
        sanitized_pitch[i] = closest_pitch_from_notes(f0[i], notes_string)
    # Perform median filtering to additionally smooth the corrected pitch.
    smoothed_sanitized_pitch = sig.medfilt(sanitized_pitch, kernel_size=11)
    # Remove the additional NaN values after median filtering.
    smoothed_sanitized_pitch[np.isnan(smoothed_sanitized_pitch)] = \
        sanitized_pitch[np.isnan(smoothed_sanitized_pitch)]
    return smoothed_sanitized_pitch

def notes_string_to_degrees(notes):
  og_notes = notes.split(',')
  new_notes = [int(note)-int(og_notes[0]) for note in og_notes]
  degrees = np.array(new_notes)
  degrees = np.concatenate((degrees, [degrees[0] + SEMITONES_IN_OCTAVE]))
  return degrees

def wav2wav_autotune(audio, pitch_audio, sr, plot=False):
    # Set some basis parameters.
    frame_length = 2048
    hop_length = frame_length // 4
    fmin = librosa.note_to_hz('C2')
    fmax = librosa.note_to_hz('C7')

    # Pitch tracking using the PYIN algorithm.
    f0, voiced_flag, voiced_probabilities = librosa.pyin(pitch_audio, frame_length=frame_length, hop_length=hop_length, sr=sr, fmin=fmin, fmax=fmax)

    # Apply the chosen adjustment strategy to the pitch.
    corrected_f0 = closest_pitch(f0)

    if plot:
        # Plot the spectrogram, overlaid with the original pitch trajectory and the adjusted
        # pitch trajectory.
        stft = librosa.stft(audio, n_fft=frame_length, hop_length=hop_length)
        time_points = librosa.times_like(stft, sr=sr, hop_length=hop_length)
        log_stft = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(log_stft, x_axis='time', y_axis='log', ax=ax, sr=sr, hop_length=hop_length, fmin=fmin, fmax=fmax)
        fig.colorbar(img, ax=ax, format="%+2.f dB")
        ax.plot(time_points, f0, label='original pitch', color='cyan', linewidth=2)
        ax.plot(time_points, corrected_f0, label='corrected pitch', color='orange', linewidth=1)
        ax.legend(loc='upper right')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [M:SS]')
        plt.savefig('pitch_correction.png', dpi=300, bbox_inches='tight')

    # Pitch-shifting using the PSOLA algorithm.
    return psola.vocode(audio, sample_rate=int(sr), target_pitch=corrected_f0, fmin=fmin, fmax=fmax)












# Librosa audio functions

def split_channels(audio_data):
  return audio_data[0], audio_data[1]

def merge_channels(left_data, right_data):
  return np.array([left_data, right_data])

def narrow_stereo(left_data, right_data, amount):
  amount = amount/2
  left = left_data * (1-amount) + right_data * amount
  right = right_data * (1-amount) + left_data * amount
  return np.array([left, right])

def time_stretch_audio(audio, sr, to_length):
  dur = librosa.get_duration(audio, sr=sr)
  return np.array([librosa.effects.time_stretch(channel, dur/to_length) for channel in split_channels(audio)])

# Slice audio signal
# Returns slices as audio
def slice_to_frames(audio_data, sr=44100, slice_duration=1, fade_in=0, fade_out=0, fx=[]):
  a_duration = librosa.get_duration(audio_data, sr=sr)
  clips = math.ceil(a_duration/slice_duration)
  frames = []
  for i in range(clips):
    # if i > 0 and i < clips:
    # if i > 0:
    start = i*slice_duration
    audio_clip = clip_audio(audio_data, sr, start, slice_duration, fx)
    frames.append( audio_clip ) #fade_audio(audio_clip, fade_in, fade_out) )
  return frames

def apply_fx():
  return

# Clip audio signal
# Returns clipped audio siangl
def clip_audio(audio_data, sr=44100, start=0, duration=10, fx=[], oneshots=False):
  xstart = librosa.time_to_samples(start, sr=sr)
  xduration = librosa.time_to_samples(start+duration, sr=sr)
  if audio_data.ndim > 1:
    audio_data = audio_data[:, xstart:xduration]
  else:
    audio_data = audio_data[xstart:xduration]
  return audio_data

# Apply fade in and/or fade out to audio signal
# Returns faded audio signal
def fade_audio(audio_data, fade_in=0, fade_out=0, sr=44100):
  a_duration = librosa.get_duration(audio_data, sr=sr)
  if fade_in > 0:
    fade_in_to = librosa.time_to_samples(fade_in, sr=sr)
    in_y = audio_data[:, 0:fade_in_to]
    fade_ins = []
    for channel in in_y:
      fade = [ i/len(channel)*smp for i, smp in enumerate(channel) ]
      fade_ins.append(fade)
    fade_ins = np.array(fade_ins)
    tail_start = fade_in_to+1  
    tail = audio_data[:, tail_start:]
    audio_data = np.concatenate([fade_ins, tail], axis=1)
  if fade_out > 0:
    fade_out_start = librosa.time_to_samples(a_duration-fade_out, sr=sr)
    out_y = audio_data[:, fade_out_start:]
    fade_outs = []
    for channel in out_y:
      fade = [ smp-(i/len(channel)*smp) for i, smp in enumerate(channel) ]
      fade_outs.append(fade)
    fade_outs = np.array(fade_outs)
    head_start = fade_out_start-1
    head = audio_data[:, :head_start]
    audio_data = np.concatenate([head, fade_outs], axis=1)
  return audio_data








def replicate(arr, times):
  return [val for val in arr for _ in range(times)]








output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.', time=True)

In [ ]:
#@title # Autotune audio to scale or nearest note { vertical-output: true, form-width: "30%" }

audio_to_autotune = ""  #@param {type: "string"}
correction_method = "scale" #@param ["scale", "nearest_note"]

#@markdown <small>Note that `scale` will be ignored if you have chosen _nearest_note_ as `correction_method` in the menu above.</small>
scale = "C:maj" #@param ['C:min', 'C#:min', 'C:maj', 'C#:maj', 'D:min', 'D#:min', 'Db:min', 'D:maj', 'D#:maj', 'Db:maj', 'E:min', 'Eb:min', 'E:maj', 'Eb:maj', 'F:min', 'F#:min', 'F:maj', 'F#:maj', 'G:min', 'G#:min', 'Gb:min', 'G:maj', 'G#:maj', 'Gb:maj', 'A:min', 'A#:min', 'Ab:min', 'A:maj', 'A#:maj', 'Ab:maj', 'B:min', 'Bb:min', 'B:maj', 'Bb:maj']



output_dir = ""  #@param {type: "string"}
stereo_width = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}

stereo = True
if stereo_width == 0:
  stereo = False


#--

uniq_id = gen_id()
input = audio_to_autotune

if os.path.isfile(drive_root+input):
  inputs = [drive_root+input]
  dir_in = path_dir(drive_root+input)
elif input != '' and os.path.isdir(drive_root+input):
  dir_in = drive_root+fix_path(input)
  # What to do if input is directory path
  inputs = list_audio(dir_in) #glob(dir_in+'/*')
elif os.path.isdir(drive_root+input) and '*' in input:
  dir_in = path_dir(drive_root+input)
  inputs = glob(drive_root+input)
else:
  op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
  sys.exit('Input not understood.')

# Output
if output_dir == '':
  dir_out = dir_in
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()
total = len(inputs)

correction_function = closest_pitch if correction_method == 'nearest_note' else partial(aclosest_pitch_from_scale, scale=scale)

# -- DO THINGS --
# print( dir_in )
# print( dir_out )

op(c.title, 'Run ID:', uniq_id)
print()

for i, input in enumerate(inputs, 1):
  ndx_info = str(i)+'/'+str(total)+' '
  op(c.title, ndx_info+'Processing', input.replace(drive_root, ''), time=True)
  print()

  file_wav_in = input
  file_wav_out = dir_out+gen_id()+'_'+str(i).zfill(3)+'.wav'

  op(c.title, 'Source audio')
  audio_player(file_wav_in)

  y, sr = librosa.load(file_wav_in, sr=None, mono=False if stereo == True else True)

  if stereo == False:
    if y.ndim > 1:
      y = y[0: :]
    pitch_corrected_y = autotune(y, sr, correction_function)
    sf.write(file_wav_out, pitch_corrected_y, sr)
    audio_player(file_wav_out)

  else:
    channel_files = []
    for i, channel in enumerate(y):
      channel_name = 'left' if i == 0 else 'right'
      tmp_wav = dir_tmp+uniq_id+'_'+channel_name+'.wav'
      pitch_corrected_y = autotune(channel, sr, correction_function)
      sf.write(tmp_wav, pitch_corrected_y, sr)

      # op(c.title, 'Result', time=True)
      # audio_player(file_wav_out)

    left, _ = librosa.load(dir_tmp+uniq_id+'_left.wav', sr=None, mono=True)
    right, _ = librosa.load(dir_tmp+uniq_id+'_right.wav', sr=None, mono=True)
    final_audio = narrow_stereo(left, right, 0.75)

    print()
    op(c.title, 'Result audio')
    sf.write(file_wav_out, final_audio.T, sr)
    
    if os.path.isfile(file_wav_out):
      audio_player(file_wav_out)
      print()
      op(c.ok, 'Autotuned WAV saved as', file_wav_out.replace(drive_root, ''), time=True)
    else:
      op(c.fail, 'ERROR saving', file_wav_out.replace(drive_root, ''), time=True)
    print()





# -- END THINGS --

timer_end = time.time()

print()
op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
op(c.ok, 'FIN.')



In [ ]:
#@title # Autotune audio to another audio { vertical-output: true, form-width: "30%" }

audio_to_autotune = ""  #@param {type: "string"}
pitch_source_audio = ""  #@param {type: "string"}

result_length = "audio_to_autotune" #@param ["audio_to_autotune", "pitch_source_audio"]

output_dir = ""  #@param {type: "string"}
stereo_width = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}

stereo = True
if stereo_width == 0:
  stereo = False


#--

uniq_id = gen_id()
input = audio_to_autotune

if os.path.isfile(drive_root+input):
  inputs = [drive_root+input]
  dir_in = path_dir(drive_root+input)
elif input != '' and os.path.isdir(drive_root+input):
  dir_in = drive_root+fix_path(input)
  # What to do if input is directory path
  inputs = list_audio(dir_in) #glob(dir_in+'/*')
elif os.path.isdir(drive_root+input) and '*' in input:
  dir_in = path_dir(drive_root+input)
  inputs = glob(drive_root+input)
else:
  op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
  sys.exit('Input not understood.')

pitch_source_audio = drive_root+pitch_source_audio
if not os.path.isfile(pitch_source_audio):
  sys.exit('pitch_source_audio should point to an audio file located in your Google Drive, e.g. audio/vocals/test.wav')

# Output
if output_dir == '':
  dir_out = dir_in
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()
total = len(inputs)



# melody = get_melody(pitch_y, pitch_sr)
# total_notes = len(melody)

# print('melody', melody)

# correction_function = partial(aclosest_pitch_from_melody, sr)

# -- DO THINGS --
# print( dir_in )
# print( dir_out )

op(c.title, 'Run ID:', uniq_id)
print()

for i, input in enumerate(inputs, 1):
  ndx_info = str(i)+'/'+str(total)+' '
  op(c.title, ndx_info+'Processing', input.replace(drive_root, ''), time=True)
  print()

  file_wav_in = input

  op(c.title, 'Source audio')
  audio_player(file_wav_in)

  op(c.title, 'Pitch audio')
  audio_player(pitch_source_audio)

  file_wav_out = dir_out+gen_id()+'_'+str(i).zfill(3)+'.wav'

  y, sr = librosa.load(file_wav_in, sr=None, mono=False if stereo == True else True)
  audio_duration = librosa.get_duration(y, sr)

  # op(c.okb, 'Estimated duration of notation:', notation_duration )
  # op(c.okb, 'Audio duration:', audio_duration )

  # if audio_duration < pitch_source_duration or timestretch_audio == 'always':

  pitch_y, pitch_sr = librosa.load(pitch_source_audio, sr=None, mono=True)
  pitch_source_duration = librosa.get_duration(pitch_y, pitch_sr)

  print()
  if result_length == "audio_to_autotune":
    pitch_y = time_stretch_audio(pitch_y, sr, audio_duration)
  else:
    op(c.okb, 'Time stretching audio to '+str(round(100*(audio_duration/pitch_source_duration), 2))+' % speed, from '+str(round(audio_duration, 4))+' to '+str(pitch_source_duration)+' seconds...', time=True)
    stretched_audio = time_stretch_audio(y, sr, pitch_source_duration)
  
  if stereo == False:
    if stretched_audio.ndim > 1:
      stretched_audio = stretched_audio[0: :]
    pitch_corrected_y = wav2wav_autotune(stretched_audio, pitch_y, sr)
    sf.write(file_wav_out, pitch_corrected_y, sr)
    audio_player(file_wav_out)

  else:
    channel_files = []
    for i, channel in enumerate(stretched_audio):
      channel_name = 'left' if i == 0 else 'right'
      tmp_wav = dir_tmp+uniq_id+'_'+channel_name+'.wav'
      pitch_corrected_y = wav2wav_autotune(channel, pitch_y, sr)
      sf.write(tmp_wav, pitch_corrected_y, sr)

      # op(c.title, 'Result', time=True)
      # audio_player(file_wav_out)

    left, _ = librosa.load(dir_tmp+uniq_id+'_left.wav', sr=None, mono=True)
    right, _ = librosa.load(dir_tmp+uniq_id+'_right.wav', sr=None, mono=True)
    final_audio = narrow_stereo(left, right, 0.75)

    print()
    op(c.title, 'Result audio')
    sf.write(file_wav_out, final_audio.T, sr)
    
    if os.path.isfile(file_wav_out):
      audio_player(file_wav_out)
      print()
      op(c.ok, 'Autotuned WAV saved as', file_wav_out.replace(drive_root, ''), time=True)
    else:
      op(c.fail, 'ERROR saving', file_wav_out.replace(drive_root, ''), time=True)
    print()





# -- END THINGS --

timer_end = time.time()

print()
op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
op(c.ok, 'FIN.')



In [ ]:
#@title # Autotune audio to [Chords Guru Turbo 100a Deluxe](https://ki.gy/cv) chord progression { vertical-output: true, form-width: "30%" }

audio_to_autotune = ""  #@param {type: "string"}
## midi_input = "temp/midilab/legnaf_2_right_131bpm_128notes_16.0s.mid"  #@param {type: "string"}

magic_string = "" #@param {type: "string"}

# chords_per_bar = 1 #@param {type:"slider", min:1, max:4, step:1}
# bpm = 120 #@param {type: "integer"}

output_dir = ""  #@param {type: "string"}
stereo_width = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}
timestretch_audio = "auto" #@param ["auto", "always"]

bpm, chords_per_bar, chords_string = magic_string.split(';')

stereo = True
if stereo_width == 0:
  stereo = False



#--

uniq_id = gen_id()
input = audio_to_autotune

if os.path.isfile(drive_root+input):
  inputs = [drive_root+input]
  dir_in = path_dir(drive_root+input)
elif input != '' and os.path.isdir(drive_root+input):
  dir_in = drive_root+fix_path(input)
  # What to do if input is directory path
  inputs = list_audio(dir_in) #glob(dir_in+'/*')
elif os.path.isdir(drive_root+input) and '*' in input:
  dir_in = path_dir(drive_root+input)
  inputs = glob(drive_root+input)
else:
  op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
  sys.exit('Input not understood.')

# Output
if output_dir == '':
  dir_out = dir_in
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()
total = len(inputs)

chords = chords_string.split('|')
total_chords = len(chords)
notation_duration = 60/bpm * (4*total_chords)

if chords_per_bar > 0:
  chords = replicate(chords, chords_per_bar)

# -- 

op(c.title, 'Run ID:', uniq_id)
print()

for i, input in enumerate(inputs, 1):
  ndx_info = str(i)+'/'+str(total)+' '
  op(c.title, ndx_info+'Processing', input.replace(drive_root, ''), time=True)
  print()
  
  file_wav_in = input

  op(c.title, 'Source audio')
  audio_player(file_wav_in)

  file_wav_out = dir_out+gen_id()+'_'+str(i).zfill(3)+'.wav'

  y, sr = librosa.load(file_wav_in, sr=None, mono=False if stereo == True else True)

  audio_duration = librosa.get_duration(y, sr)

  # op(c.okb, 'Estimated duration of notation:', notation_duration )
  # op(c.okb, 'Audio duration:', audio_duration )

  if audio_duration < notation_duration or timestretch_audio == 'always':
    print()
    op(c.okb, 'Time stretching audio to '+str(round(100*(audio_duration/notation_duration), 2))+' % speed, from '+str(round(audio_duration, 4))+' to '+str(notation_duration)+' seconds...', time=True)
    stretched_audio = time_stretch_audio(y, sr, notation_duration)
    stretched_audio_slices = slice_to_frames(y, sr, notation_duration/total_chords)
  else:
    stretched_audio_slices = [y]

  total_audio_clips = len(stretched_audio_slices)
  # op(c.okb, 'Processing '+str(total_audio_clips)+' audio frames...', time=True)

  all_audio_slices = []

  for ii, audio_slice in enumerate(stretched_audio_slices):
    ndx_info_ii = str(ii+1)+'/'+str(total_audio_clips)+' '
    # op(c.okb, ndx_info_ii+'Processing slice...', time=True)
    slice_y = audio_slice

    slice_wav_out = dir_tmp+uniq_id+'_slice_'+str(i).zfill(3)+'_'+str(ii).zfill(3)+'.wav'
    correction_function = partial(aclosest_pitch_from_notes, notes_string=chords[ii])

    if stereo == False:
      if slice_y.ndim > 1:
        slice_y = slice_y[0: :]
      pitch_corrected_y = autotune(slice_y, sr, correction_function)
      sf.write(slice_wav_out, pitch_corrected_y, sr)
      # audio_player(slice_wav_out)

    else:
      channel_files = []
      for iii, channel in enumerate(slice_y):
        channel_name = 'left' if iii == 0 else 'right'
        slice_wav_out = dir_tmp+uniq_id+'_slice_'+str(i).zfill(3)+'_'+str(ii).zfill(3)+'_'+channel_name+'.wav'
        pitch_corrected_y = autotune(channel, sr, correction_function)
        sf.write(slice_wav_out, pitch_corrected_y, sr)

        # op(c.title, 'Result', time=True)
        # audio_player(file_wav_out)

      left, _ = librosa.load(dir_tmp+uniq_id+'_slice_'+str(i).zfill(3)+'_'+str(ii).zfill(3)+'_left.wav', sr=None, mono=True)
      right, _ = librosa.load(dir_tmp+uniq_id+'_slice_'+str(i).zfill(3)+'_'+str(ii).zfill(3)+'_right.wav', sr=None, mono=True)
      final_slice = narrow_stereo(left, right, stereo_width)
      sf.write(slice_wav_out, final_slice.T, sr)
      if os.path.isfile(slice_wav_out):
        # audio_player(slice_wav_out)
        all_audio_slices.append(final_slice)
      else:
        op(c.fail, 'Error saving', slice_wav_out.replace(drive_root, ''), time=True)

  audio_concat = np.concatenate(all_audio_slices, axis=1)

  print()
  op(c.title, 'Result audio')
  sf.write(file_wav_out, audio_concat.T, sr)
  
  if os.path.isfile(file_wav_out):
    audio_player(file_wav_out)
    print()
    op(c.ok, 'Autotuned WAV saved as', file_wav_out.replace(drive_root, ''), time=True)
  else:
    op(c.fail, 'ERROR saving', file_wav_out.replace(drive_root, ''), time=True)
  print()

  # print('test 1')
  # sf.write('test1.wav', audio_concat.T, sr)
  # audio_player('test1.wav')

  # sf.write(file_wav_out, concat_y.T, sr)
  # audio_player(file_wav_out)










# -- END THINGS --

timer_end = time.time()

print()
op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
op(c.ok, 'FIN.')

